<i><b>Public AI</b></i>
<br>
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 2. Convolution Neural Network**
# Section 7. Fully Convolutional Network Segmentation(modified)


In [ ]:
# If You use in Colab, You Should run this script
import os
if (not os.path.exists("./multicampus_segmentation_generator") and
    not "multicampus_segmentation_generator" in os.getcwd()):
    !git clone https://github.com/public-ai/multicampus_segmentation_generator.git
    os.chdir("./multicampus_segmentation_generator")

In [ ]:
from tensorflow.keras.layers import Layer, Conv2D, MaxPool2D, Conv2DTranspose, ZeroPadding2D
from tensorflow.keras.layers import Input, UpSampling2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.layers import Softmax, Add
from tensorflow.keras.layers import Lambda, concatenate
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import optimizers
import tensorflow.keras.backend as K 
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt 
from keras.utils import np_utils
import numpy as np
import tensorflow as tf
import cv2
import sys 
sys.path.append('../')
from generator import fasion_mnist_generator

# Load Dataset

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fasion_mnist_generator(1000, 10)
print('Train Image shape : ', train_images.shape)
print('Train Labels shape : ', train_labels.shape)
print('Test Image shape : ', test_images.shape)
print('Test Labels shape : ', test_labels.shape)

In [ ]:
fig, axes = plt.subplots(1,2)
fig.set_size_inches((10,10))
sample_image = train_images[0]
axes[0].imshow(train_images[0])
axes[0].set_title('Image')

sample_mask = np.argmax(train_labels[0], axis=-1)
axes[1].imshow(sample_mask)
axes[1].set_title('Label')

plt.show()

# Model

![Imgur](https://i.imgur.com/tp5MGGu.png)

In [ ]:
tf.reset_default_graph()

# 위 그림에 만족하는 Convolution Neural Network를 구성하고 모델을 생성해주세요
# fix me!

# Training

In [ ]:
# metric 
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [ ]:
# compile & fitting 
sgd = optimizers.SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=[mean_iou])
results = model.fit(train_images/255., train_labels, validation_split=0.1, batch_size=16, epochs=20)

# Test 

In [ ]:
pred = model.predict(test_images[:100]/255.)
pred_cls = np.argmax(pred, axis=-1)

In [ ]:
fig, axes = plt.subplots(2,10)
ognl_axes = np.array(axes)[0, :].flatten()
pred_axes = np.array(axes)[1, :].flatten()
fig.set_size_inches(30,10)
for i in range(10):
    ognl_axes[i].imshow(test_images[i])
    pred_axes[i].imshow(pred_cls[i] == 0)

---
⊙ Copyright(c) 2020 by PublicAI. All rights reserved <br>
All pictures, codes, writings cannot be copied without permission. <br>
Writen by PAI(info@publicai.co.kr) <br>
last updated on 2020/01/4 <br>

---